In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import csv


loco_url = "https://traveloco.jp/"
id_url = "https://traveloco.jp/korea?p="
df = pd.DataFrame(columns=["Id","Question", "Content", "Answer"])
id_list=[]


In [2]:
def find_id(id_html,id_list):
    bsObject = BeautifulSoup(id_html, "html.parser") 
    cnt=0
    for link in bsObject.find_all('a'):
        find_loco = "/loco/"
        del_word ="intervie"
        temp = link.get('href')
        if cnt==0 :
            if (temp[0:6] == find_loco) & (temp[-9:-1]!=del_word) :
                id_list.append(temp)
                cnt=1
        else :
            cnt=0
        pass


In [3]:
def find_page(URL) :
    maximum = 1
#    URL = 'https://traveloco.jp/loco/rqykd5222/q?p=1'
    html = urlopen(URL)  
    bsObject = BeautifulSoup(html, "html.parser") 


    page_object = bsObject.find(class_='pagination')

    if page_object != None :
        page_href=page_object.find_all('a')
        for link in page_href:#.find('a'):
            maximum = link.get('href')
            pass

    if maximum == 1 :
        return 1
    else :
        return int(maximum[3:])

In [4]:
def find_content(id):
    
    #init
    _df = pd.DataFrame(columns=["Id","Question", "Content", "Answer"])
    
    cnt=0
    page_min=1
    url="https://traveloco.jp/loco/"+id+"/q?p="
    #page 찾기
    page_max = find_page(url+"1")
    
    #for문으로 돌리면서 확인
    
    for idx in range(page_max) :
        cur_page=idx+1
        test = urlopen(url+str(cur_page))
        bsObject = BeautifulSoup(test, "html.parser") 
        test.close()
        title=[]
        question=[]
        answer=[]
        for link in bsObject.find_all('a'):
            if (link.get('href')[0:6] == "/loco/") & (len(link.get('href'))>len("id")+8):
                if cnt==0 :
                    title.append(link.text.strip())
                    cnt=1
                else :
                    cnt=0
            pass

        for question_ in bsObject.find_all('p',class_='loco_question_content') :
            question.append(question_.text.strip())
            pass

        for answer_ in bsObject.find_all('p',class_='loco_answer_content') :
            answer.append(answer_.text.strip())
            pass

        if len(question) == len(answer) :
            for i in range(len(question)) :
                temp_df = pd.DataFrame(data=[[id,title[i+1],question[i],answer[i]]],columns=["Id","Question", "Content", "Answer"])
                _df=_df.append(temp_df)

    return _df

In [5]:
#한국인 로코 아이디 

csv_file = open("../Result/id_list.csv","r",newline="")
tt=csv.reader(csv_file)

for line in tt:
    id_list = line
csv_file.close()

if len(id_list) == 0 :
    for i in range(37):
        id_html = urlopen("https://traveloco.jp/korea?p="+str(i+1))  
        find_id(id_html,id_list)
        pass
    
    #save
    csv_file = open("../Result/id_list.csv","w",newline="")
    csvwriter = csv.writer(csv_file)
    # for row in id_list:
    csvwriter.writerow(id_list)
    csv_file.close()

    id_list
    

In [ ]:
df = pd.DataFrame(columns=["Id","Question", "Content", "Answer"])
idx=0
cnt=0

for id in id_list :
    
    print(idx,id)
    page=1
    
    temp_df = find_content(id[6:])
    df=df.append(temp_df)#,columns=["Id","Question", "Content", "Answer"])
    if idx==99:
        #저장
        df=df.reset_index(drop=True)
        df.to_csv("../Result/total_"+str(cnt)+".csv")
        cnt=cnt+1
        
        #init
        df = pd.DataFrame(columns=["Id","Question", "Content", "Answer"])
    
    idx=(idx+1) % 100
    pass



0 /loco/rqykd5222
